In [ ]:
# 역할 분담

# 파일 로드 - 제품설명서 + FAQ 목록 -> vectorDB 에 넣을 거
## 가져와서 metadata 랑 chunking 전처리하고 vectorDB에 넣기까지

# RAG 구조 + 모델링 - 메타 데이터 및 쿼리 재생성 + vectorDB에서 검색
## SQL Agent 까지 여기서 구성 

# 개인정보용 SQLDataBase 구성 - 고객 정보 테이블, 고객 장바구니?, 고객 주문내역
## 해당 정보 관련 query 일 경우 별도의 tool로 개인정보 가져오기

# RAG 평가 지표 구성
## RAGAS 이용, 지표 골라야함

In [ ]:
# 요구 패키지
%pip install unstructured[md]
%pip install bs4

In [4]:
# BeautifulSoup 을 이용하여 FAQ 추출
## 제품설명서도 따와야함 
import requests
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from uuid import uuid4

documents = []

for i in range(6):
    # 파싱 과정
    url = f"https://www.musinsa.com/app/cs/faq/00{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    result_tag = soup.select("body > main > section > ul > li > button > em")
    result_title = soup.select("body > main > section > ul > li > button > p")
    result_content = soup.select("body > main > section > ul > li > div > p")

    print(len(result_tag), len(result_title), len(result_content))

    # Document 타입으로 저장
    for j in range(len(result_tag)):
        doc = Document(
            page_content=str(result_content[j]),
            metadata={"source":"FAQ", 
                    "tag":str(result_tag[j]), 
                    "title":str(result_title)},
            id=str(uuid4())
            )
        documents.append(doc)
print(len(documents), '\n', documents[0])

15 15 15
11 11 11
14 14 14
14 14 14
18 18 18
69 69 69
141 
 page_content='<p>보유한 적립금이 소멸되기 전 메일로 내용을 안내드리고 있습니다.<br/>
적립금 소멸 안내 상황은 아래에서 확인해 주세요. <br/>
<br/>
■ 적립금 소멸 안내 상황<br/>
1. 보유 적립금 유효기간 만료 예정 <br/>
- 보유 중인 적립금의 유효기간이 만료되기 30일 전, 소멸 예정 안내 메일이 발송됩니다. <br/>
※ 적립금은 유효기간 내 사용 시 정상적으로 이용하실 수 있습니다.<br/>
<br/>
2. 1년 이상 미이용 시 <br/>
- 무신사 스토어 이용약관에 따라, 1년 이상 로그인 이력이 없는 경우 적립금이 소멸됩니다. <br/>
- 11개월간 로그인하지 않은 상태에서 적립금을 보유한 회원에게  적립금 일괄 소멸 30일 전 안내 메일이 발송됩니다. <br/>
※ 적립금 소멸을 원하지 않으실 경우, 소멸 예정일 이전에 로그인해 주세요.</p>' metadata={'source': 'FAQ', 'tag': '<em class="faq-list__category">탈퇴/기타</em>', 'title': '[<p class="faq-list__question">적립금 소멸 예정 메일을 받았어요.</p>, <p class="faq-list__question">회원 탈퇴를 취소하고 싶습니다.</p>, <p class="faq-list__question">회원 탈퇴는 어떻게 하나요?</p>, <p class="faq-list__question">소셜 로그인(카카오, Apple) 연동을 해제하고 싶어요. 어떻게 하면 되나요?</p>, <p class="faq-list__question">애플(Apple) 로그인 및 애플(Apple) 계정 연동이 안 됩니다.</p>, <p class="faq-list__question">소셜 로그인은 어떻게 이용하나요?</p>, <p class="faq-list__question">아

In [ ]:
documents[0].metadata

{'source': 'FAQ',
 'tag': '<em class="faq-list__category">탈퇴/기타</em>',
 'title': '[<p class="faq-list__question">적립금 소멸 예정 메일을 받았어요.</p>, <p class="faq-list__question">회원 탈퇴를 취소하고 싶습니다.</p>, <p class="faq-list__question">회원 탈퇴는 어떻게 하나요?</p>, <p class="faq-list__question">소셜 로그인(카카오, Apple) 연동을 해제하고 싶어요. 어떻게 하면 되나요?</p>, <p class="faq-list__question">애플(Apple) 로그인 및 애플(Apple) 계정 연동이 안 됩니다.</p>, <p class="faq-list__question">소셜 로그인은 어떻게 이용하나요?</p>, <p class="faq-list__question">아이디와 비밀번호가 기억나지 않아요.</p>, <p class="faq-list__question">아이디 및 비밀번호를 변경할 수 있나요?</p>, <p class="faq-list__question">회원 정보 수정은 어디서 하나요?</p>, <p class="faq-list__question">휴대폰 번호 인증 시 이미 존재하는 휴대폰이라고 뜰 때는 어떻게 해야 하나요?</p>, <p class="faq-list__question">본인인증 없이 상품을 구매할 수 있나요?</p>, <p class="faq-list__question">본인인증을 잘못했는데 초기화 가능한가요?</p>, <p class="faq-list__question">본인인증 문자가 오지 않아요.</p>, <p class="faq-list__question">본인인증은 어떻게 하나요?</p>, <p class="faq-list__question">회원 가입은 어떻게 하나요?</p>]'}

In [5]:
# Document 들 저장 - Json
## 내부 내용 다듬기는 해야함
import json
import os

os.makedirs("data", exist_ok=True)

documents_dict = []
for doc in documents:
    doc_dict = {"page_content": doc.page_content, "metadata": doc.metadata, "id": doc.id}
    documents_dict.append(doc_dict)
dir_name = "../data/"
file_name = "documents.json"
with open(os.path.join(dir_name, file_name), "w", encoding="utf-8") as f:
    json.dump(documents_dict, f, ensure_ascii=False, indent=2)
len(doc_dict), doc_dict

(3,
 {'page_content': '<p><span style="font-size:14px;">■ 적립금이란?<br/>\n상품을 구매하거나 구매한 상품의 후기 작성, 이벤트 참여 등을 통해 지급됩니다.<br/>\n* 지급된 적립금은 상품 구매 시, 결제 금액의 최대 7%까지 사용할 수 있습니다.<br/>\n* 적립금은 10원 단위로만 사용 가능합니다.<br/>\n* 주문 시 적립금 대신 적립금 선할인을 통해 할인을 받을 수도 있습니다. (단, 현금으로 환급은 불가)<br/>\n* 제휴(광고) 접속 여부 또는 일부 상품은 구매 적립 및 적립금 선할인이 제한될 수 있습니다.<br/>\n<br/>\n■ 적립금 확인하기<br/>\n- 모바일(앱/웹) : 마이 &gt; 적립금에서 확인<br/>\n<br/>\n※ 적립금 전환 : 포인트 제도 변경으로 2024년 7월 28일부터 포인트를 적립금으로 전환할 수 없습니다.<br/>\n<br/>\n<strong><a href="https://www.musinsa.com/mypage/legacy/point"><span style="color:#3498db;">포인트 적립 확인 바로가기</span></a><br/>\n<a href="https://www.musinsa.com/app/cs/notice_list#no_14497"><span style="color:#3498db;">포인트 제도 일부 변경 안내 바로가기</span></a></strong></span></p>',
  'metadata': {'source': 'FAQ',
   'tag': '<em class="faq-list__category">혜택</em>',
   'title': '[<p class="faq-list__question">무신사 스탠다드 익스프레스는 뭔가요?</p>, <p class="faq-list__question">오프라인 구매 상품 후기는 어떻게 작성하나요?</p>, <p class="faq-list__question">후기 규정을 알려주세요.<

In [ ]:
# Chunking 과정 

In [6]:
# 임베딩 + Vector Store 구성
## FAQ_db 랑 instructions_db 두 개 구성하기 
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_chroma import Chroma

COLLECTION_NAME = "product_usage"
PERSISTENT_PATH = "../data/vector_store/chroma/FAQ_db"

# Vector Store 생성
def get_vector_store():
    load_dotenv()
    embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')

    vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSISTENT_PATH
    )
    return vector_store
vector_store = get_vector_store()

# 문서 추가 
vector_store.add_documents(documents)
vector_store._collection.count()

INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


141